# Step 2: Private Prediction using Syft Keras - Serving (Server)

In [1]:
from collections import OrderedDict
#import torch
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe

In [2]:
from syft.keras import KerasHook

In [3]:
hook = KerasHook(tf.keras)

In [4]:
dense = tf.keras.layers.Dense(2)

(2,)
starred 2
{}


In [5]:
dense._syft_args_store

(2,)

In [6]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

## Protocol

In [7]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [8]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [9]:
for player_name in players.keys():
    print("python -m tf_encrypted.player --config /tmp/tfe.config {}".format(player_name))

python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2


## Model

In [10]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [11]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=task_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

(10, (3, 3))
starred 10 (3, 3)
{'batch_input_shape': [1, 28, 28, 1]}
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


((2, 2),)
starred (2, 2)
{}
('relu',)
starred relu
{}
(32, (3, 3))
starred 32 (3, 3)
{}
((2, 2),)
starred (2, 2)
{}
('relu',)
starred relu
{}
(64, (3, 3))
starred 64 (3, 3)
{}
((2, 2),)
starred (2, 2)
{}
('relu',)
starred relu
{}
()
starred
{}
(10,)
starred 10
{'name': 'logit'}


In [12]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [13]:
data = np.ones([1, 28, 28, 1])

In [14]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[  8.768467   -8.874218    3.5424995  -4.6425986   1.0684487   0.9196919
   17.240221  -20.799267    4.18496    -9.126283 ]]


### Secure the model by sharing the weights

In [15]:
prot = tfe.protocol.SecureNN()

tfe_graph, tfe_model = model.share(prot=prot)

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_input InputLayer
conv2d Conv2D
FullArgSpec(args=['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint'], varargs=None, varkw='kwargs', defaults=((1, 1), 'valid', None, None, None, True, 'glorot_uniform', 'zeros', None, None, None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={})
parameters ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint', 'kwargs']
arg_list ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer'

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_1 Conv2D
FullArgSpec(args=['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint'], varargs=None, varkw='kwargs', defaults=((1, 1), 'valid', None, None, None, True, 'glorot_uniform', 'zeros', None, None, None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={})
parameters ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint', 'kwargs']
arg_list ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularize

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_2 Conv2D
FullArgSpec(args=['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint'], varargs=None, varkw='kwargs', defaults=((1, 1), 'valid', None, None, None, True, 'glorot_uniform', 'zeros', None, None, None, None, None), kwonlyargs=[], kwonlydefaults=None, annotations={})
parameters ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularizer', 'kernel_constraint', 'bias_constraint', 'kwargs']
arg_list ['self', 'filters', 'kernel_size', 'strides', 'padding', 'data_format', 'dilation_rate', 'activation', 'use_bias', 'kernel_initializer', 'bias_initializer', 'kernel_regularizer', 'bias_regularizer', 'activity_regularize

## Computation

**Note computation, tfe.serving.QueueServer etc. will move into model.share()**

In [ ]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

def computation(x):
    return tfe_model(x)

In [ ]:
with tfe_graph.as_default():

    server = tfe.serving.QueueServer(
        input_shape=input_shape,
        output_shape=output_shape,
        computation_fn=computation)
    
    initializer = tf.global_variables_initializer()

### Serving

In [ ]:
sess = tfe.Session(graph=tfe_graph)

tf.Session.reset(sess.target)

sess.run(initializer, tag='init')

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



In [ ]:
def step_fn():
    print("Next")

server.run(
    sess,
    num_steps=3,
    step_fn=step_fn)